# Number of photons in laser pulse, as a function of the number of laser slices

_________

#### Imports

In [ ]:
import sys, time
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal
import rslaser.utils.srwl_uti_data as srwutil

import scipy.constants as const
from scipy import special

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

### Load File Data

In [ ]:
package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data')
meta_file_name = 'wfs_meta.dat'
ccd_name = 'ccd_pump_off.txt'
wfs_name = 'wfs_pump_off.txt'
meta_path_to_file = os.path.join(package_data_dir, meta_file_name)
ccd_path_to_file = os.path.join(package_data_dir, ccd_name)
wfs_path_to_file = os.path.join(package_data_dir, wfs_name)

files = PKDict(
    meta = meta_path_to_file,
    ccd = ccd_path_to_file,
    wfs = wfs_path_to_file
    )

## From Function

In [ ]:
num_slices = 6
crystal_params = PKDict(
    length = 2.5/100.0,  # [m]
    nslice = 6,
    n0          = [1.76 for _ in range(num_slices)],
    n2          = [16.0 for _ in range(num_slices)],
    pump_energy = 0.035,  # [J]
    pump_waist  = 1.2e-3 /1.18,  # [m]
)

prop_type = 'gain_calc'

max_num_l_slices = 50

total_n_photons_i = np.zeros(max_num_l_slices)
total_n_photons   = np.zeros(max_num_l_slices)

for num_l_slices in np.arange(max_num_l_slices):

    params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = num_l_slices+1,
        nx_slice = 32,
        ny_slice = 32,
        pulseE = 1.0e-6,
        tau_fwhm = 300.0e-12 / math.sqrt(2.), #0.1 / const.c / math.sqrt(2.),
        sigx_waist = 1.2e-3 /1.18, #1.0e-3,
        sigy_waist = 1.2e-3 /1.18, #1.0e-3,
    )
    
    e_crystal = crystal.Crystal(crystal_params)
    thisPulse = pulse.LaserPulse(params)
    nslices_pulse = len(thisPulse.slice)
    nslices_crystal = len(e_crystal.slice)
    
    for laser_index_i in np.arange(nslices_pulse):
        total_n_photons_i[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh))
    
    if (num_l_slices < max_num_l_slices):
        #print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

    for laser_index_p in np.arange(nslices_pulse):
        total_n_photons[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh))

n_l_slice = np.arange(1,max_num_l_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_l_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Laser Slices in Pulse')

n_l_slice_func = n_l_slice
total_n_photons_func = total_n_photons_i
thisPulse_func = thisPulse

In [ ]:
if thisPulse.slice[0].ds == thisPulse.slice[-1].ds:
    fig_x = thisPulse.slice[0].ds *(np.arange(nslices_pulse) + 0.5)
else:
    print('Slices are different lengths!')

nslices_pulse = len(thisPulse.slice)
photons_in_slice = np.zeros((nslices_pulse))
for laser_index in np.arange(nslices_pulse):
        photons_in_slice[laser_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d.mesh))

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(fig_x,photons_in_slice,'k')
box = ax.get_position()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')

x_function = fig_x
photons_function = photons_in_slice

## From File

In [ ]:
crystal_params = PKDict(length = 2.5/100.0,  # [m]
                        nslice = 5,
)
prop_type = 'gain_calc'

max_num_l_slices = 50

total_n_photons_i = np.zeros(max_num_l_slices)
total_n_photons   = np.zeros(max_num_l_slices)

for num_l_slices in np.arange(max_num_l_slices):

    params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = num_l_slices+1,
        nx_slice = 32,
        ny_slice = 32,
        pulseE = 1.0e-6,
        tau_fwhm = 300.0e-12 / math.sqrt(2.), #0.1 / const.c / math.sqrt(2.),
        sigx_waist = 1.2e-3 /1.18, #1.0e-3,
        sigy_waist = 1.2e-3 /1.18, #1.0e-3,
    )
    
    e_crystal = crystal.Crystal(crystal_params)
    thisPulse = pulse.LaserPulse(params, files)
    nslices_pulse = len(thisPulse.slice)
    nslices_crystal = len(e_crystal.slice)
    
    for laser_index_i in np.arange(nslices_pulse):
        total_n_photons_i[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_i].n_photons_2d.mesh))
    
    if (num_l_slices < max_num_l_slices):
        #print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

    for laser_index_p in np.arange(nslices_pulse):
        total_n_photons[num_l_slices] += np.sum(np.sum(thisPulse.slice[laser_index_p].n_photons_2d.mesh))

n_l_slice = np.arange(1,max_num_l_slices + 1)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice,total_n_photons_i,'k',label='after initialization')
plt.plot(n_l_slice,total_n_photons,'--k',label='after propagation')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Laser Slices in Pulse')

n_l_slice_file = n_l_slice
total_n_photons_file = total_n_photons_i
thisPulse_file = thisPulse

In [ ]:
if thisPulse.slice[0].ds == thisPulse.slice[-1].ds:
    fig_x = thisPulse.slice[0].ds *(np.arange(nslices_pulse) + 0.5)
else:
    print('Slices are different lengths!')

nslices_pulse = len(thisPulse.slice)
photons_in_slice = np.zeros((nslices_pulse))
for laser_index in np.arange(nslices_pulse):
        photons_in_slice[laser_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d.mesh))

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(fig_x,photons_in_slice)
box = ax.get_position()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')

x_file = fig_x
photons_file = photons_in_slice

***
# Comparison

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))
ax2 = ax1.twinx()
ax1.plot(x_file,photons_file, 'k', label='File Init')
ax2.plot(x_function,photons_function, '--r', label='Function Init')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax1.tick_params(direction="in")
ax2.tick_params(direction="in")
ax1.set_xlabel(r'Distance from Laser Front [m]')
ax1.set_ylabel(r'Number of Photons, File Init')
ax2.set_ylabel(r'Number of Photons, Function Init',color='r')
plt.title('Longitudinal Photon Distribution')

In [ ]:
print(np.mean(total_n_photons_file/total_n_photons_func))

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
plt.plot(n_l_slice_file,total_n_photons_file,'k',label='File Init')
plt.plot(n_l_slice_func,total_n_photons_func,'r',label='Function Init')
plt.legend()
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Number of Laser Slices in Pulse')

In [ ]:
# Plot function wavefront
wfr = thisPulse_func.slice[0].wfr

x=np.linspace(wfr.mesh.xStart,wfr.mesh.xFin,wfr.mesh.nx)
y=np.linspace(wfr.mesh.yStart,wfr.mesh.yFin,wfr.mesh.ny)

e_total = thisPulse_func.extract_total_2d_elec_fields()
intensity_func = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(x*(1e3), y*(1e3), intensity_func, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (Function Init)')

In [ ]:
# Plot function wavefront
wfr = thisPulse_file.slice[0].wfr

x=np.linspace(wfr.mesh.xStart,wfr.mesh.xFin,wfr.mesh.nx)
y=np.linspace(wfr.mesh.yStart,wfr.mesh.yFin,wfr.mesh.ny)

e_total = thisPulse_file.extract_total_2d_elec_fields()
intensity_file = 0.5 *const.c *const.epsilon_0 *(e_total.re**2.0 + e_total.im**2.0)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(6,3.6))
    ax = fig.gca()
    plt.pcolormesh(x*(1e3), y*(1e3), intensity_file, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (File Init)')

In [ ]:
print('Pulse energy, file init: ',thisPulse_file.pulseE)
print('Pulse energy, function init: ',thisPulse_func.pulseE)

In [ ]:
thisPulse = pulse.LaserPulse(params, files)
energy_2d = 0

nslices_pulse = len(thisPulse.slice)
for laser_index_i in np.arange(nslices_pulse):
    wfr = thisPulse.slice[laser_index_i].wfr
    
    x=np.linspace(wfr.mesh.xStart,wfr.mesh.xFin,wfr.mesh.nx)
    y=np.linspace(wfr.mesh.yStart,wfr.mesh.yFin,wfr.mesh.ny)
    cell_area = np.mean(np.diff(x)) *np.mean(np.diff(y))
        
    # total component of electric field
    re0, re0_mesh = srwutil.calc_int_from_wfr(wfr, _pol=6, _int_type=5, _det=None, _fname='', _pr=False)
    im0, im0_mesh = srwutil.calc_int_from_wfr(wfr, _pol=6, _int_type=6, _det=None, _fname='', _pr=False)
    
    e_total_re = np.array(re0).reshape((wfr.mesh.nx, wfr.mesh.ny), order='C').astype(np.float64)
    e_total_im = np.array(im0).reshape((wfr.mesh.nx, wfr.mesh.ny), order='C').astype(np.float64)
    
    efield_abs_sqrd_2d = (e_total_re**2.0 + e_total_im**2.0)
    
    end1 = (thisPulse.slice[laser_index_i]._pulse_pos -0.5*thisPulse.slice[laser_index_i].ds) /(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)
    end2 = (thisPulse.slice[laser_index_i]._pulse_pos +0.5*thisPulse.slice[laser_index_i].ds) /(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)
    
    energy_2d += cell_area *(const.epsilon_0 /2.0) \
                *(efield_abs_sqrd_2d /np.exp(-thisPulse.slice[laser_index_i]._pulse_pos**2.0/(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)**2.0)) \
                *((np.sqrt(np.pi)/2.0) *(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s) *(special.erf(end2) -special.erf(end1)))

print('Total energy_2d, file init: ', np.sum(energy_2d))

thisPulse = pulse.LaserPulse(params)
energy_2d = 0

nslices_pulse = len(thisPulse.slice)
for laser_index_i in np.arange(nslices_pulse):
    wfr = thisPulse.slice[laser_index_i].wfr
    
    x=np.linspace(wfr.mesh.xStart,wfr.mesh.xFin,wfr.mesh.nx)
    y=np.linspace(wfr.mesh.yStart,wfr.mesh.yFin,wfr.mesh.ny)
    cell_area = np.mean(np.diff(x)) *np.mean(np.diff(y))
        
    # total component of electric field
    re0, re0_mesh = srwutil.calc_int_from_wfr(wfr, _pol=6, _int_type=5, _det=None, _fname='', _pr=False)
    im0, im0_mesh = srwutil.calc_int_from_wfr(wfr, _pol=6, _int_type=6, _det=None, _fname='', _pr=False)
    
    e_total_re = np.array(re0).reshape((wfr.mesh.nx, wfr.mesh.ny), order='C').astype(np.float64)
    e_total_im = np.array(im0).reshape((wfr.mesh.nx, wfr.mesh.ny), order='C').astype(np.float64)
    
    efield_abs_sqrd_2d = (e_total_re**2.0 + e_total_im**2.0)
    
    end1 = (thisPulse.slice[laser_index_i]._pulse_pos -0.5*thisPulse.slice[laser_index_i].ds) /(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)
    end2 = (thisPulse.slice[laser_index_i]._pulse_pos +0.5*thisPulse.slice[laser_index_i].ds) /(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)
    
    energy_2d += cell_area *(const.epsilon_0 /2.0) \
                *(efield_abs_sqrd_2d /np.exp(-thisPulse.slice[laser_index_i]._pulse_pos**2.0/(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s)**2.0)) \
                *((np.sqrt(np.pi)/2.0) *(np.sqrt(2.0) *thisPulse.slice[laser_index_i].sig_s) *(special.erf(end2) -special.erf(end1)))

print('Total energy_2d, function init: ', np.sum(energy_2d))

In [ ]:
print('Expected number of photons, 1uJ pulse: ', (1.0e-6 *800.0e-9)/(const.h *const.c))